In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.9 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os, shutil
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/data'
base_dir = '/content/drive/MyDrive/data'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
conv_base = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 1024)              52429824  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 8)                 8200      
                                                                 
Total params: 74,240,808
Trainable params: 74,206,376
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch = 10

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)
label_map = (train_generator.class_indices)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)

label_map

Found 23219 images belonging to 8 classes.
Found 2585 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [ ]:
train_stop = 3604//batch
test_stop = 401//batch

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=[tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),
                       tfa.metrics.F1Score(num_classes=8),
                       tfa.metrics.FBetaScore(num_classes=8)])

In [ ]:
%%time
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=10)

Epoch 1/10
360/360 [==============================] - 1315s 4s/step - loss: 23.2368 - categorical_accuracy: 0.2867 - recall: 0.2186 - precision: 0.3553 - f1_score: 0.2679 - fbeta_score: 0.2679
Epoch 2/10
360/360 [==============================] - 1081s 3s/step - loss: 2.2990 - categorical_accuracy: 0.2731 - recall: 0.1436 - precision: 0.5764 - f1_score: 0.2414 - fbeta_score: 0.2414
Epoch 3/10
360/360 [==============================] - 933s 3s/step - loss: 2.0569 - categorical_accuracy: 0.2881 - recall: 0.1497 - precision: 0.6849 - f1_score: 0.2521 - fbeta_score: 0.2521
Epoch 4/10
360/360 [==============================] - 824s 2s/step - loss: 1.9962 - categorical_accuracy: 0.2956 - recall: 0.1614 - precision: 0.7516 - f1_score: 0.2620 - fbeta_score: 0.2620
Epoch 5/10
360/360 [==============================] - 738s 2s/step - loss: 1.9170 - categorical_accuracy: 0.3022 - recall: 0.1661 - precision: 0.7513 - f1_score: 0.2482 - fbeta_score: 0.2482
Epoch 6/10
360/360 [======================

In [ ]:
test = model.evaluate(test_generator, steps=test_stop)

40/40 [==============================] - 156s 4s/step - loss: 1.7228 - categorical_accuracy: 0.3825 - recall: 0.1625 - precision: 0.8553 - f1_score: 0.3376 - fbeta_score: 0.3376


In [ ]:
model.save('/content/drive/MyDrive/dataset/InceptionV3.h5')

In [ ]:
import pandas as pd

In [ ]:
df_marks = pd.DataFrame({'x_test': test_generator.filepaths,
     'y_test': test_generator.labels})

x_test = df_marks.drop(['y_test'],axis=1)
y_test = df_marks['y_test']



In [ ]:
pred = model.predict(test_generator)

259/259 [==============================] - 845s 3s/step


In [ ]:
predicted_classes = np.argmax(pred, axis = 1)
print(predicted_classes)

[0 0 0 ... 3 5 0]


In [ ]:
from sklearn.metrics import classification_report


In [ ]:
report = classification_report(y_test, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       0.13      0.56      0.21       322
           1       0.10      0.04      0.06       325
           2       0.29      0.01      0.01       321
           3       0.11      0.10      0.10       326
           4       0.10      0.01      0.02       322
           5       0.13      0.13      0.13       321
           6       0.10      0.02      0.04       325
           7       0.12      0.11      0.12       323

    accuracy                           0.12      2585
   macro avg       0.13      0.12      0.09      2585
weighted avg       0.13      0.12      0.09      2585

